In [7]:
import pandas as pd
import os
from jphrase import extracter

path = os.path.dirname(os.path.abspath("__file__")) 
df_texts = pd.read_table(path + "/text.tsv", header=None, lineterminator='\n', names=["sentence"])
jpex = extracter()
jpex.get_dfphrase(df_texts)




途中経過


,seqchar,length,freq,sc_index,knowns
2,キャンペーン,6.0,79.0,6.173786,0.0
1,YouTube,7.0,26.0,5.241747,0.0
0,Twitter,7.0,20.0,4.990433,0.0
6,名様にプレゼント,8.0,14.0,4.787492,0.0
3,プレゼントキャンペーン,11.0,9.0,4.700480,0.0


concat終了 -> ソーティング -> セレクション -> 重複除去


,seqchar,length,freq,sc_index,knowns,hiragana,katakana,hankana,kanji,alphabet,smalla,largea,zalpha,number,znumber,start_what,end_what,periodic,originality
0,キャンペーン,6.0,153.0,6.828712,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,1.000000
1,プレゼントキャンペーン,11.0,23.0,5.575949,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,0.588235
2,インタビュー,6.0,8.0,3.988984,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,1.000000
3,YouTube,7.0,78.0,6.315358,False,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,alphabet,alphabet,0.0,1.000000
4,Twitter,7.0,52.0,5.916202,False,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,alphabet,alphabet,0.0,1.000000
5,NintendoSwitch,14.0,12.0,5.204007,False,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,alphabet,alphabet,0.0,1.000000
6,NAVIGATOR,9.0,16.0,5.030438,False,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,alphabet,alphabet,0.0,1.000000
7,OPENREC,7.0,16.0,4.779123,False,0.0,0.0,0.0,0.0,2.0,0.0,2.0,0.0,0.0,0.0,alphabet,alphabet,0.0,1.000000
8,SixTONES,8.0,12.0,4.644391,False,0.0,0.0,0.0,0.0,2.0,1.0,1.0,0.0,0.0,0.0,alphabet,alphabet,0.0,0.933333
11,名様にプレゼント,8.0,39.0,5.768321,False,1.0,1.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,0.0,kanji,katakana,0.0,0.947368


In [8]:

print("パラメータを指定して、フレーズの検知を始めます。")
params = {}#ハイパーパラメータのデフォルト値を設定
params["verbose"] = 1
params["size_sentence"] = 5000#一回で処理するセンテンスの数：大きすぎると計算が終わらない
params["min_count"] = 10#文字連カウントの最小数閾値：小さくすると計算終わらない
params["min_length"] = 4 #文字の長さ最小値
params["max_length"] = 16#文字の長さ最大値
params["weight_freq"] = 1 #頻度への重み（たくさんある連なりを重視）
params["weight_len"] = 2   #長さへの重み（長い連なりを重視）
params["removes"] = "⠀ #�\n.：.…!！？?･ｰ￣*～\r\u3000、/＼／「」『』【】"#走査前に除去する文字
params["unnecesary"] = ["http", "www", "ｗｗｗ", "&amp;", "&gt;"]#走査後に除去する文字列
params["selection"] = 1# 0: セレクションの有無
params["positive_filter"] = ["_2","__2", "___2","____2TZ___",#英単語（半角英文字のみ、ただし大文字が１つ以上入る  
                            "_______2__",#全角英文字のみ
                            "_1_10___00","__110___00",#カナ漢（情報コミュニケーション学、途中で挟まるひらがなは構わない、数字の混入は認めない） #ｶﾅ漢
                            "00011___00","0001___100",#漢英#漢全英
                            ]
params["negative_filter"] = {"start":[0,8,9,-1], "end":[0,8,9,-1],#ひらがな/数字 での開始or終了
                            "periodic": True, "smalla":True, "kanantsu":True, "less_than_maxlen":True }
                            #周期性は不要、ひらがなや数字の開始終了も不要 　文字連の最大値は不使用
params["threshold_originality"] = 0.60#独自性の閾値（.0にすれば全く絞らず、.9なら順位の低い似たフレーズが除去される）
params["knowns"] = []


jpex = extracter(**params)
jpex.get_dfphrase(df_texts)


パラメータを指定して、フレーズの検知を始めます。
途中経過


,seqchar,length,freq,sc_index,knowns
15,キャンペーン,6.0,115.0,8.337109,0.0
36,フォロー,4.0,219.0,8.166216,0.0
37,プレゼント,5.0,133.0,8.116716,0.0
38,プレゼントキャンペーン,11.0,18.0,7.740230,0.0
31,ツイート,4.0,132.0,7.662938,0.0


concat終了 -> ソーティング -> セレクション -> 重複除去


,seqchar,length,freq,sc_index,knowns,hiragana,katakana,hankana,kanji,alphabet,smalla,largea,zalpha,number,znumber,start_what,end_what,periodic,originality
0,キャンペーン,6.0,153.0,8.620472,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,1.000000
1,フォロー,4.0,311.0,8.515592,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,1.000000
2,プレゼント,5.0,196.0,8.502080,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,katakana,0.0,1.000000
3,ツイート,4.0,189.0,8.019613,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,katakana,0.0,0.888889
6,ディズニー,5.0,32.0,6.715383,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,0.888889
7,チャンネル,5.0,31.0,6.684612,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,katakana,0.0,0.727273
9,イベント,4.0,42.0,6.533789,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,katakana,0.0,0.666667
10,チェック,4.0,36.0,6.383507,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,katakana,0.0,0.888889
11,ポケモン,4.0,36.0,6.383507,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,kanantsu,0.0,0.888889
12,オリジナル,5.0,22.0,6.354370,False,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,katakana,katakana,0.0,0.800000
